In [2]:
import numpy as np
#script to generate the 2Q files (Q present in two directions where the second direction is obtained by 90 degree 
#rotation around the y-axis). The structures of increasing Q were obtained from amplimodes on the bilbao crystallographic server
#and subsequently converted to vasp format. Vasp format was used for ease - no calculations were done with VASP!


#permutation matrix of corresponding atoms after rotation
rot_index = np.empty(32)
rot_index[0]=4
rot_index[1]=1
rot_index[2]=2
rot_index[3]=7
rot_index[4]=0
rot_index[5]=5
rot_index[6]=6
rot_index[7]=3
rot_index[8]=12
rot_index[9]=9
rot_index[10]=10
rot_index[11]=15
rot_index[12]=8
rot_index[13]=13
rot_index[14]=14
rot_index[15]=11
rot_index[16]=28
rot_index[17]=27
rot_index[18]=26
rot_index[19]=29
rot_index[20]=24
rot_index[21]=31
rot_index[22]=30
rot_index[23]=25
rot_index[24]=18
rot_index[25]=17
rot_index[26]=20
rot_index[27]=23
rot_index[28]=22
rot_index[29]=21
rot_index[30]=16
rot_index[31]=19

#roation matrix around y axis
angle = 90 * 2*np.pi/360
ROTY = np.zeros((3,3))
ROTY[0,0]=np.cos(angle)
ROTY[0,2]=np.sin(angle)
ROTY[1,1]=1
ROTY[2,0]=-np.sin(angle)
ROTY[2,2]=np.cos(angle)

#load the original coordinates
xyz_orig1 = np.loadtxt("Q_0.vasp", skiprows=8)
#rotate them by 90 degrees around y
xyz_orig2 = np.empty(xyz_orig1.shape)
for i in range(0, xyz_orig1.shape[0]):
    xyz_orig2[i] = ROTY.dot(np.transpose(xyz_orig1[i]))

#POSCAR header
poscar_header = """
1.0
        7.3236460686 0 0
        0 7.7495899200 0
        0 0 7.3236460686
    W    O
    8   24
Direct
"""

#check if there are doubles in the permutation array
for i in range(0, rot_index.size):
    comp = rot_index[i]
    for j in range(0, rot_index.size):
        if not i == j and comp == rot_index[j]:
            print("doubles in permutation matrix")

#construct the 4fold rotation around y permutation matrix
perm_matrix = np.empty((rot_index.size, rot_index.size))
for i in range(0, rot_index.size):
    for j in range(0, rot_index.size):
        if j == rot_index[i]:
            perm_matrix[i,j] = 1
        else:
            perm_matrix[i,j] = 0
            
#get the original atom coordinates without the presence of the order parameter and shift the z components by 0.25
xyz_orig1 = np.loadtxt("Q_0.vasp", skiprows=8)
xyz_orig1[:,2] += 0.25
#and the original atoms rotated by 90 degrees around the y axis
xyz_orig2 = np.empty(xyz_orig1.shape)
for i in range(0, xyz_orig1.shape[0]):
    xyz_orig2[i] = ROTY.dot(np.transpose(xyz_orig1[i]))
    
for i in range(0, 41):
    #get the displaced coordinates in the first direction
    xyz_dir1 = np.loadtxt("Q_" + str(i) + ".vasp", skiprows=8)
    #shift the z components by 0.25 to make a and c faces equivalent
    xyz_dir1[:,2] += 0.25
    #get the displacements in the first direction
    dxyz_dir1 = xyz_orig1 - xyz_dir1
    
    #get the displacements in the second direction by rotation around the y axis by 90 degrees
    dxyz_dir2 = np.empty(xyz_orig1.shape)
    for j in range(0, dxyz_dir1.shape[0]):
        dxyz_dir2[j] = ROTY.dot(np.transpose(dxyz_dir1[j]))        
        
    #add the displacements in the first direction to the original structure
    new_xyz = np.copy(xyz_orig1)
    new_xyz += dxyz_dir1
  
    #add the displacements in the second direction to the original but permute them accordingly first
    dxyz_dir2 = perm_matrix.dot(dxyz_dir2)
    new_xyz += dxyz_dir2
    
    #make the poscar file
    poscar_file = poscar_header
    for j in range(0, xyz_orig1.shape[0]):
        for k in range(0, 3):
            poscar_file += str(new_xyz[j,k]) + '\t'
        poscar_file += '\n'
    
    new_file = "2Q_" + str(i) + ".vasp"
    new_file = open(new_file, 'w')
    new_file.write(poscar_file)
    new_file.close()

print("done")

done
